# Automatically add article/journal info to a Google Scholar profile
Export your Google Scholar profile as CSV,

upload the file(s) from [here](./),

This program will add new columns like:
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>Institution Authors</th>
      <th>Institution Groups</th>
      <th>DOI</th>
      <th>ISSN</th>
      <th>Impact Factor</th>
      <th>Quartil</th>
      <th>Journal Hindex</th>
      <th>...</th>
    </tr>
  </thead>
</table>
The enriched CSV file can be saved and used as a [Spreadsheet](https://docs.google.com/spreadsheets/d/19dx09hBthKknZs1XxIKuGgwq3ieyPGx8Xr3hi4xqvBc/edit?usp=sharing).

Full Code in this public [repository](https://github.com/restrepo/articles).

To run the program use `<Shift>+<Enter>` in each Code Cell (or from the Menu: `Cell -> Run Cells`)

In [1]:
from publications import *

Here we assume that there exist a data base with the authors multiple names associated to the real name in the format:
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>Author_Names</th>
      <th>Control</th>
      <th>Full_Name</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td> Zapata, Oscar;Zapata, O.</td>
      <td> 0</td>
      <td> Óscar Alberto Zapata Noreña</td>
    </tr>
  </tbody>
</table>
Where control is some identification number associated to the author.


We assume also that there is a data base with the Group at which some real name author belongs in the format:
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>Full_Name</th>
      <th>Institution_Group</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td> Óscar Alberto Zapata Noreña</td>
      <td> GFIF</td>
    </tr>
  </tbody>
</table>

In [2]:
a=articles('citations.csv',user='-6mndWkAAAAJ',authors_file='authors.json',group_file='groups.json')

In [3]:
fulldoi=a.articles_update()

Updating entry:0.1.2.3.4.5.6.7.8.9.10.11.12.

In [4]:
a.articles

,Authors,Title,Publication,Volume,Number,Pages,Year,Publisher,Institution_Authors,Institution_Groups,DOI,ISSN,DOI_Journal,Impact_Factor,Quartil,Journal_Hindex
0,"Sierra, D Aristizabal; Kubo, Jisuke; Suematsu, Daijiro; Restrepo, D; Zapata, Oscar;","Radiative seesaw model: Warm dark matter, collider signatures, and lepton flavor violating signals",Physical Review D,79,1,13011,2009,APS,Óscar Alberto Zapata Noreña,GFIF,http://dx.doi.org/10.1103/physrevd.79.013011,1550-7998;1550-2368,Physical Review D,4.643,Q1,259
1,"Restrepo, Diego; Taoso, Marco; Valle, JWF; Zapata, Oscar;",Gravitino dark matter and neutrino masses with bilinear R-parity violation,Physical Review D,85,2,23523,2012,APS,Óscar Alberto Zapata Noreña,GFIF,http://dx.doi.org/10.1103/physrevd.85.023523,1550-7998;1550-2368,Physical Review D,4.643,Q1,259
2,"Choi, Ki-Young; Restrepo, Diego; Yaguna, Carlos E; Zapata, Oscar;",Indirect detection of gravitino dark matter including its three-body decays,Journal of Cosmology and Astroparticle Physics,2010,10,33,2010,IOP Publishing,Óscar Alberto Zapata Noreña,GFIF,http://dx.doi.org/10.1088/1475-7516/2010/10/033,1475-7516,Journal of Cosmology and Astroparticle Physics,5.81,Q3,70
3,"Sierra, D Aristizabal; Restrepo, Diego; Zapata, Oscar;",Decaying neutralino dark matter in anomalous U (1) H models,Physical Review D,80,5,55010,2009,APS,Óscar Alberto Zapata Noreña,GFIF,http://dx.doi.org/10.1103/physrevd.80.055010,1550-7998;1550-2368,Physical Review D,4.643,Q1,259
4,"Klasen, Michael; Yaguna, Carlos E; Ruiz-Álvarez, José D; Restrepo, Diego; Zapata, Oscar;",Scalar dark matter and fermion coannihilations in the radiative seesaw model,Journal of Cosmology and Astroparticle Physics,2013,4,44,2013,IOP Publishing,Óscar Alberto Zapata Noreña,GFIF,http://dx.doi.org/10.1088/1475-7516/2013/04/044,1475-7516,Journal of Cosmology and Astroparticle Physics,5.81,Q3,70
5,"Florez, Andres; Restrepo, Diego; Velasquez, Mauricio; Zapata, Oscar;",Baryonic violation of R parity from anomalous U (1) H,Physical Review D,87,9,95010,2013,APS,Óscar Alberto Zapata Noreña,GFIF,http://dx.doi.org/10.1103/physrevd.87.095010,1550-7998;1550-2368,Physical Review D,4.643,Q1,259
6,"Molinaro, Emiliano; Yaguna, Carlos E; Zapata, Oscar;",FIMP realization of the scotogenic model,Journal of Cosmology and Astroparticle Physics,2014,7,15,2014,IOP Publishing,Óscar Alberto Zapata Noreña,GFIF,http://dx.doi.org/10.1088/1475-7516/2014/07/015,1475-7516,Journal of Cosmology and Astroparticle Physics,5.81,Q3,70
7,"Restrepo, Diego; Rivera, Andrés; Sánchez-Peláez, Marta; Zapata, Oscar; Tangarife, Walter;",Radiative neutrino masses in the singlet-doublet fermion dark matter model with scalar singlets,Physical Review D,92,1,13005,2015,APS,Óscar Alberto Zapata Noreña,GFIF,http://dx.doi.org/10.1103/physrevd.92.013005,1550-7998;1550-2368,Physical Review D,4.643,Q1,259
8,"Arbeláez, Carolina; Longas, Robinson; Restrepo, Diego; Zapata, Oscar;",Fermion dark matter from SO (10) GUTs,Physical Review D,93,1,13012,2016,APS,Óscar Alberto Zapata Noreña,GFIF,http://dx.doi.org/10.1103/physrevd.93.013012,2470-0010;2470-0029,Physical Review D,4.643,,
9,"Ponce, William A; Zapata, Oscar;",Lepton masses and mixing without Yukawa hierarchies,Physical Review D,74,9,93007,2006,APS,Óscar Alberto Zapata Noreña,GFIF,http://dx.doi.org/10.1103/physrevd.74.093007,1550-7998;1550-2368,Physical Review D,4.643,Q1,259


In [6]:
fulldoi.ix[8]

DOI                                                                                                                                                                                              10.1103/physrevd.93.013012
ISSN                                                                                                                                                                                                 [2470-0010, 2470-0029]
URL                                                                                                                                                                            http://dx.doi.org/10.1103/physrevd.93.013012
article-number                                                                                                                                                                                                       013012
author              [{'affiliation': [], 'given': 'Carolina', 'family': 'Arbeláez'}, {'affiliation': [], 'given': 'Robin

## Save uptated data base

In [5]:
a.to_csv('newcitations.csv')

Restore saved data base

In [6]:
arts=articles('newcitations.csv',authors_file='authors.json',group_file='groups.json')

In [7]:
arts.articles_update()

Updating entry:0.1.2.

""


In [8]:
arts.articles

,Authors,Title,Publication,Volume,Number,Pages,Year,Publisher,Institution_Authors,Institution_Groups,DOI,ISSN,DOI_Journal,Impact_Factor,Quartil,Journal_Hindex
0,"Sierra, D Aristizabal; Kubo, Jisuke; Suematsu, Daijiro; Restrepo, D; Zapata, Oscar;","Radiative seesaw model: Warm dark matter, collider signatures, and lepton flavor violating signals",Physical Review D,79,1,13011,2009,APS,Óscar Alberto Zapata Noreña,GFIF,http://dx.doi.org/10.1103/physrevd.79.013011,1550-7998;1550-2368,Physical Review D,4.643,Q1,259
1,"Restrepo, Diego; Taoso, Marco; Valle, JWF; Zapata, Oscar;",Gravitino dark matter and neutrino masses with bilinear R-parity violation,Physical Review D,85,2,23523,2012,APS,Óscar Alberto Zapata Noreña,GFIF,http://dx.doi.org/10.1103/physrevd.85.023523,1550-7998;1550-2368,Physical Review D,4.643,Q1,259
2,"Choi, Ki-Young; Restrepo, Diego; Yaguna, Carlos E; Zapata, Oscar;",Indirect detection of gravitino dark matter including its three-body decays,Journal of Cosmology and Astroparticle Physics,2010,10,33,2010,IOP Publishing,Óscar Alberto Zapata Noreña,GFIF,http://dx.doi.org/10.1088/1475-7516/2010/10/033,1475-7516,Journal of Cosmology and Astroparticle Physics,5.810,Q3,70


[Develompent notebook](./dev.ipynb)